In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## DTLN Model Training

In [ ]:
%cd /content
!git clone https://github.com/WujuMaster/DTLN.git

In [ ]:
!pip install wavinfo

In [ ]:
!unrar x -o+ "./data/DTLN/8khz/training_set.part1.rar" "/content/"
!unrar x -o+ "./data/DTLN/8khz/training_set.part2.rar" "/content/"
!unrar x -o+ "./data/DTLN/8khz/training_set.part3.rar" "/content/"

In [ ]:
!du -sh /content/training_set 

In [ ]:
%cd /content/DTLN
!python run_training.py

In [ ]:
%cd /content/DTLN
!python run_evaluation.py -i /content/data -o /content/output -m ./pretrained_model/DTLN_model.h5

# Model testing

In [ ]:
%cd /content
!git clone https://github.com/WujuMaster/DTLN.git

In [ ]:
!pip install wavinfo
# !pip install tensorflow==2.3

In [ ]:
%cd /content/DTLN
!python convert_weights_to_saved_model.py -m /content/DTLN/pretrained_model/DTLN_model.h5 -t /content/drive/MyDrive/GGColab/Save/DTLN/SavedModel_16khz/

In [ ]:
%cd /content/DTLN/
!python convert_weights_to_onnx.py -m ./pretrained_model/DTLN_model_233epoch.h5 -t /content/drive/MyDrive/GGColab/Save/DTLN/Onnx/my_model

In [ ]:
%cd /content/DTLN/
!python convert_weights_to_tf_lite.py -q True -m ./pretrained_model/DTLN_model.h5 -t ./pretrained_model/DTLN_8khz_model

In [ ]:
!cp /content/DTLN/my_model_1.tflite /content/drive/MyDrive/GGColab/Save/DTLN/tf-lite
!cp /content/DTLN/my_model_2.tflite /content/drive/MyDrive/GGColab/Save/DTLN/tf-lite

In [ ]:
!apt-get install rar

In [ ]:
!rar a '/content/drive/MyDrive/GGColab/Save/DTLN/dtln_saved_model' '/content/DTLN/pretrained_model/dtln_saved_model'

In [ ]:
!rar a '/content/drive/MyDrive/GGColab/Save/DTLN/DTLN_norm_500h_saved_model' '/content/DTLN/pretrained_model/DTLN_norm_500h_saved_model'

## TFJS conversion

In [ ]:
%cd /content
!git clone https://github.com/WujuMaster/DTLN.git

In [ ]:
!pip install wavinfo
!pip install tensorflow==2.3
!pip install tensorflowjs==2.3.0

In [ ]:
!pip show tensorflowjs

In [ ]:
%cd /content/DTLN

/content/DTLN


In [ ]:
import tensorflowjs as tfjs
import tensorflow as tf

tf.keras.backend.set_learning_phase(0)
tfjs.converters._enable_tflite_resource_variables = True

!tensorflowjs_converter --input_format=tf_saved_model \
                         --output_node_names='test_model' \
                         --saved_model_tags=serve \
                         /content/drive/MyDrive/GGColab/Save/DTLN/SavedModel/ \
                         /content/drive/MyDrive/GGColab/Save/DTLN/

In [ ]:
import tensorflowjs as tfjs
from DTLN_model import DTLN_model

model_class = DTLN_model()
model_class.build_DTLN_model_stateful()
model_class.model.load_weights('./pretrained_model/DTLN_model.h5')
tfjs.converters.save_keras_model(model_class.model, 'DTLN_js')